# Prebuilt ReAct Agent

This notebook demonstrates how to create and customize a prebuilt ReAct agent using LangChain and LangGraph.

This notebook is divided into three main sections:

1. **Configuration**: Setting up the required packages and API keys
   - Installation of necessary packages (langchain-openai, langgraph, langchain-community)
   - Configuration of OpenAI, Tavily, and Langchain API keys
   - Langchain API keys are used for Langsmith tracing / observability

2. **Tool Use**: Creating and implementing tools for the agent
   - Basic tool implementation with stock price lookup
   - Integration with Tavily Search
   - Customization of agent behavior with system prompts
   - Advanced state management and prompt templates

3. **Memory Management**: Adding persistence to agent conversations
   - Implementation of thread-based chat memory
   - Demonstration of context retention across multiple queries

## 1. Configuration

To start, we install the required packages. These packages provide essential tools and integrations:

- langchain-openai: Adds OpenAI API functionality to LangChain.
- langgraph: Offers pre-built agents and tools for creating more advanced AI agents.
- langchain-community: Contains community-contributed tools and modules to extend LangChain’s capabilities.

In [ ]:
!pip install langchain-openai==0.2.0
!pip install langgraph==0.2.23
!pip install langchain-community==0.3.1

In [2]:
# from dotenv import load_dotenv
# load_dotenv()

We set the API keys as environment variables using getpass to prompt for input.

In [ ]:
import os
import getpass

# openai key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAi Key here:")

# tavily key
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily Key here:")


In [ ]:
# langchain key for langsmith tracing / observability
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Langchain Key here:")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ODSC_Workshop"

## 2. Tool Use

Here, we import essential modules:

ChatOpenAI: Provides access to the OpenAI language model.
create_react_agent: A prebuilt agent from langgraph that allows interactive conversations.
StructuredTool: A tool in LangChain that lets you define structured functions that an agent can call.

In [4]:
from datetime import datetime
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain.tools import StructuredTool


We define a custom get_current_stock_price function that provides fixed responses for [company]. This function is wrapped as a StructuredTool, which allows the agent to use it. We then initialize the ChatOpenAI model and create a reactive agent (graph) that listens to user inputs and responds using the get_current_stock_price tool.

In [ ]:
# define dummy tool function
def get_current_stock_price(company: str) -> str:
    '''Return the current stock price for the specified company.'''
    if 'nvidia' in company.lower():
        stock_price = '135.37'
    elif 'amd' in company.lower():
        stock_price = '153.44'
    elif 'intel' in company.lower():
        stock_price = '23.20'
    else:
        stock_price = 'unknown'
    
    return f"The current stock price of {company} is ${stock_price}"

tools = [StructuredTool.from_function(get_current_stock_price)]
model = ChatOpenAI(model="gpt-4o")
graph = create_react_agent(model, tools=tools)

inputs = {"messages": [("user", "What is the current stock price of NVIDIA")]}
for s in graph.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

This cell demonstrates the use of Tavily Search. We define a search tool, TavilySearchResults, to retrieve the market cap and stock price for NVIDIA. After initializing and running it, we store the results in tools for later use with the agent.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# define search tool

search = TavilySearchResults(max_results=2)
search_results = search.invoke("What is the market cap and stock price for NVIDIA")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

Here, we customize the agent with a system prompt instructing it to respond to the users question and add buy/sell commentary.

In [ ]:
# Adding a system prompt to the agent
system_prompt = """You are an expert financial analyst that specializes in analyzing 
technology companies. You answer the user question, and then provide a buy / sell 
recommendation and justification for your recommendation."""

graph = create_react_agent(model, tools, state_modifier=system_prompt)
inputs = {"messages": [("user", "What is the market cap and stock price for NVIDIA on Dec 4 2024")]}
for s in graph.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()


Here we are setting up a LangChain conversation graph that uses a custom prompt template and state management to create an AI agent that can process messages and tools. The agent is then run with an initial message asking about the market cap and stock price for NVIDIA, with custom state variables including today's date, message history, and a step flag.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

# Add complex prompt with custom graph state
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Today is {today}"),
        ("placeholder", "{messages}"),
    ]
)
class CustomState(TypedDict):
    today: str
    messages: Annotated[list[BaseMessage], add_messages]
    is_last_step: str

# Create the graph
graph = create_react_agent(model, tools, state_schema=CustomState, state_modifier=prompt)

# Define the inputs
inputs = {"messages": [("user", "What is the market cap and stock price for NVIDIA")], "today": datetime.now().strftime("%B %d, %Y"), "is_last_step": False}

# Stream the outputs
for s in graph.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

## 3. Add memory to the agent

We can add persistent chat memory to the conversation graph using MemorySaver, allowing the agent to remember previous interactions within a specific thread. It then demonstrates this by running two sequential queries where the second query ("What is the buy / sell recommendation") can reference context from the first query about NVIDIA.

In [ ]:
# add thread level chat-memory to the graph

from langgraph.checkpoint.memory import MemorySaver
graph = create_react_agent(model, tools, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "thread-1"}}
def print_stream(graph, inputs, config):
    for s in graph.stream(inputs, config, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
inputs = {"messages": [("user", "What is the market cap and stock price for NVIDIA on Dec 4 2024")]}
print_stream(graph, inputs, config)
inputs2 = {"messages": [("user", "What is the buy / sell recommendation")]}
print_stream(graph, inputs2, config)

